# CO2 prediction model

#### Read in data and imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
data =  pd.read_csv("data/cleanedData.csv")

#### Need to remove vehicles without CO2 emissions

In [3]:
df_NoCO2 = data[data.CO2.isna()]
df_CO2 = data[data.CO2.notnull()]

#### Make all features numeric

In [4]:
y = df_CO2['CO2']
X = df_CO2.drop(columns=['CO2'])
y = pd.get_dummies(y)
X = pd.get_dummies(X)

#### Split the data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#### Classifiers

In [6]:
#model = RandomForestClassifier(criterion='gini', random_state = 0).fit(X_train, y_train)
#model = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)

In [7]:
acc = accuracy_score(y_test, model.predict(X_test))
print(acc)

0.8226427494767222


KNeighbornClassifier(n_neihbors=5) Accuracy = 82.26%